###Dataset used::
The dataset taken in this problem statement is the famous GTZAN music genre dataset.

GTZAN dataset has 10 categories of music genre like blues, classical, pop etc. 

Each 10 categories have 100 data samples in .wav format.

In [1]:
!wget "http://opihi.cs.uvic.ca/sound/genres.tar.gz"

--2020-12-13 12:30:42--  http://opihi.cs.uvic.ca/sound/genres.tar.gz
Resolving opihi.cs.uvic.ca (opihi.cs.uvic.ca)... 142.104.68.135
Connecting to opihi.cs.uvic.ca (opihi.cs.uvic.ca)|142.104.68.135|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1225571541 (1.1G) [application/x-gzip]
Saving to: ‘genres.tar.gz’

genres.tar.gz       100%[===================>]   1.14G   879KB/s    in 22m 56s 

2020-12-13 12:53:39 (870 KB/s) - ‘genres.tar.gz’ saved [1225571541/1225571541]



In [2]:
!tar -xvf genres.tar.gz

genres/
genres/bextract_single.mf
genres/bl.mf
genres/blues/
genres/cl.mf
genres/classical/
genres/co.mf
genres/country/
genres/di.mf
genres/disco/
genres/hi.mf
genres/hiphop/
genres/input.mf
genres/ja.mf
genres/jazz/
genres/me.mf
genres/metal/
genres/po.mf
genres/pop/
genres/re.mf
genres/reggae/
genres/ro.mf
genres/rock/
genres/rock/rock.00000.wav
genres/rock/rock.00001.wav
genres/rock/rock.00002.wav
genres/rock/rock.00003.wav
genres/rock/rock.00004.wav
genres/rock/rock.00005.wav
genres/rock/rock.00006.wav
genres/rock/rock.00007.wav
genres/rock/rock.00008.wav
genres/rock/rock.00009.wav
genres/rock/rock.00010.wav
genres/rock/rock.00011.wav
genres/rock/rock.00012.wav
genres/rock/rock.00013.wav
genres/rock/rock.00014.wav
genres/rock/rock.00015.wav
genres/rock/rock.00016.wav
genres/rock/rock.00017.wav
genres/rock/rock.00018.wav
genres/rock/rock.00019.wav
genres/rock/rock.00020.wav
genres/rock/rock.00021.wav
genres/rock/rock.00022.wav
genres/rock/rock.00023.wav
genres/rock/rock.00024.wav
g

In [3]:
#importing required libraries
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import os
from PIL import Image
import pathlib
import csv

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

import keras
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers import Dense
from keras.optimizers import Adam

import warnings
warnings.filterwarnings('ignore')

In [4]:
#header name for the csv file 
header = 'filename'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [5]:
#extracting mfcc features and writing into csv file
file = open('dataset.csv', 'w', newline='') 
with file:
    writer = csv.writer(file)
    writer.writerow(header)

genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'genres/{g}'):
        songname = f'genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)  
        to_append = f'{filename}' 
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('dataset.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [6]:
#reading the csv file
data = pd.read_csv('dataset.csv')
data.head()

,filename,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00018.wav,-252.533986,104.927485,15.353578,42.656638,6.778399,21.618894,-1.299683,5.684369,-16.614657,12.166094,-1.953308,1.960557,-0.881435,7.778604,2.390255,1.774140,1.339317,3.630699,4.910860,-2.651944,blues
1,blues.00083.wav,-19.867150,98.897808,-24.477810,49.591759,-16.202193,30.204456,-18.029301,27.562145,-11.405339,14.883301,-8.911968,11.674803,-10.887061,9.315449,-5.833091,9.261364,-4.050388,3.255899,-3.466852,2.380785,blues
2,blues.00005.wav,-177.869048,118.196916,-17.550673,30.758635,-21.742746,11.903802,-20.734249,3.180597,-8.583484,-0.936488,-11.776273,-2.420615,-9.339365,-9.939325,-3.909892,-5.570625,-1.839023,-2.778421,-3.046866,-8.115809,blues
3,blues.00025.wav,-249.281996,130.997924,0.665435,66.696115,17.905895,4.622960,6.610036,1.245376,1.419175,-1.584293,-0.974159,-3.575065,-2.926915,4.598686,7.478115,-0.178770,-2.818857,-2.667636,0.751667,-0.533179,blues
4,blues.00055.wav,-98.674753,104.795413,-35.109565,60.701628,-2.831404,22.236794,-7.836521,19.030045,-11.581366,17.020950,-5.386074,12.202055,-9.961467,0.357868,-4.265253,6.134887,-4.830016,1.503692,-1.491254,3.509777,blues


In [23]:
#segregating the filename from the dataset
data = data.drop(['filename'],axis=1)

#encoding the Labels
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

#scaling the Feature columns
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

KeyError: ignored

In [24]:
#splitting data into training and Testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [8]:
genre_list.nunique()

10

In [9]:
print("Training X shape: ",X_train.shape)
print("Training Y shape: ",y_train.shape)

Training X shape:  (800, 20)
Training Y shape:  (800,)


In [10]:
print("Test X shape: ",X_test.shape)
print("Test Y shape: ",y_test.shape)

Test X shape:  (200, 20)
Test Y shape:  (200,)


In [25]:
X_train=X_train.reshape(800,20,1)

In [26]:
X_train.shape

(800, 20, 1)

In [27]:
#LSTM Model
model = Sequential()
model.add(LSTM(128, activation='relu', dropout=0.05, return_sequences=True, input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(LSTM(32, activation='relu', dropout=0.05, return_sequences=False))
model.add(Dense(10, activation="softmax"))

In [28]:
model.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=["accuracy"])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 20, 128)           66560     
_________________________________________________________________
lstm_5 (LSTM)                (None, 32)                20608     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                330       
Total params: 87,498
Trainable params: 87,498
Non-trainable params: 0
_________________________________________________________________


In [29]:
classifier=model.fit(X_train, y_train, epochs=100, batch_size=64)

Epoch 1/100


ValueError: ignored